In [4]:
import numpy as np
import pandas as pd
import re
import jieba
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
#加载文本
original_data = pd.read_excel('/content/gdrive/My Drive/dataset/te_data.xlsx')
#加载停用词
stopwords = [line.strip() for line in open('/content/gdrive/My Drive/dataset/stopwords.txt', 'r', encoding='utf-8').readlines()]

In [42]:
#清洗数据
data = original_data
data.drop_duplicates(subset=['知识内容'],inplace=True)#dropping重复
data.dropna(axis=0,inplace=True)#dropping空值
data['摘要']=np.where(data['知识内容']!='',0,data['知识内容'])

In [7]:
#文本预处理：去除特殊字符，分句子
def pre_proc(text): 
  text = re.sub(r'[[0-9]*]',' ',text)#去除类似[1]，[2]
  text = re.sub(r'[。）]','）',text)#去除'。）'里的'。'
  text = re.sub(r'\s+',' ',text)#用单个空格替换了所有额外的空格
  sentences = re.split('(。|！|\!|\.)',text)#分句
  return sentences

#词频
def freq(text):
  word2count = {} #line 1
  for word in jieba.cut(text): #对整个文本分词
    if word not in stopwords:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1
  for key in word2count.keys():
    word2count[key] = word2count[key] / max(word2count.values())
  return word2count


#计算句子得分
def score(sentences,word2count):
  sent2score = {}
  for sentence in sentences:
    for word in jieba.cut(sentence):
        if word in word2count.keys():
            if len(sentence)<300:
                if sentence not in sent2score.keys():
                    sent2score[sentence] = word2count[word]
                else:
                    sent2score[sentence] += word2count[word]# bug:如果句子很长，分数就高
  return sent2score

#字典排序
def dic_order_value_and_get_key(dicts, count):
    # by hellojesson
    # 字典根据value排序，并且获取value排名前几的key
    final_result = []
    # 先对字典排序
    sorted_dic = sorted([(k, v) for k, v in dicts.items()], reverse=True)
    # tmp_set = set()  
    # for item in sorted_dic:
    #     tmp_set.add(item[1])
    # for list_item in sorted(tmp_set, reverse=True)[:count]:# 用于选择前几名
    for i,dic_item in enumerate(sorted_dic):# 选择句子得分大于20的句子，和排名第一的句子
            if dic_item[1] > 20:   
              final_result.append(dic_item[0])
            elif i<1:
              final_result.append(dic_item[0])
    return final_result

In [60]:
#摘要输出
for i,text in enumerate(data.iloc[:,1]):
  sentences = pre_proc(text)
  word2count = freq(text)
  sent2score = score(sentences,word2count)
  final_resul=dic_order_value_and_get_key(sent2score,0)
  data.iloc[i,2] = '。'.join(final_resul)

In [61]:
data.to_excel("/content/gdrive/My Drive/dataset/jieba_answer.xlsx", sheet_name='jieba')  